<a href="https://colab.research.google.com/github/Julioore/TFM/blob/main/TFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Valoración de opciones financieras
* Método Binomial
* Método de Black-Scholes
* Convergencia entre ambos métodos
* Gráfico de convergencia

In [ ]:
import numpy as np                              # importamos las librerías
from scipy.stats import binom
from scipy.stats import norm
import matplotlib as mpl
import matplotlib.pyplot as plt

# Fórmula para método de valoración opciones europeas mediante árbol binomial

def prima(tipo, spot, strik, anyos, volatilidad, n, tInt):
    r = np.log(1+tInt)                          # r es el tanto instantáneo
    deltaT = anyos / n
    u = np.exp(volatilidad * np.sqrt(deltaT))
    d = 1/u
    p = (np.exp(r*deltaT)-d) / (u-d)
    Sn = [0]*(n+1)                              # Sn es la lista de precios del activo subyacene a fecha de vto.
    payoff = [0]*(n+1)                          # inicilizamos a cero la lista del payoff
    prima = 0
    for i in range(n+1):
        Sn[i] = spot * u**(n-i) * d**i
        if tipo.lower() == 'call':
            payoff[i] = max(0, Sn[i] - strik)
        if tipo.lower() == 'put':
            payoff[i] = max(0, strik - Sn[i])
        prima += payoff[i] * binom.pmf(n-i, n, p)
    prima *= np.exp(-r * anyos)                 # descontamos a valor actual, hasta t=0
    return prima

# Fórmula para método de Black-Scholes

def primaBS(tipo, spot, strik, anyos, volatilidad, tInt):
    r = np.log(1+tInt)                          # r es el tanto instantáneo
    d1 = (np.log(spot/strik) + (r + 0.5 * volatilidad**2) * anyos) / (volatilidad * np.sqrt(anyos))
    d2 = d1 - volatilidad * np.sqrt(anyos)
    if tipo.lower() == 'call':
      primaBS = spot * norm.cdf(d1) - strik * np.exp(-r * anyos) * norm.cdf(d2)
    if tipo.lower() == 'put':
      primaBS = strik * np.exp(-r * anyos) * norm.cdf(-d2) - spot * norm.cdf(-d1)
    return primaBS

# Representamos gráficamente la valoración por el método binomial
# Cuando el número de pasos del método binomial tiende a infinito el 
# valor teórico obtenido coincide con el que proporciona el método de Black-Scholes

m = 200                                   # Calcularemos el arbol desde 1 hasta m pasos
evolucion = [0]*(m)
for j in range(m):
    evolucion[j] = prima('call', 100, 110, 2, 0.2, j+1, 0.07)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
last = 1
bound = m
ax.plot(evolucion)
fig.savefig('graph.png')
print('Prima(1)=', evolucion[0])
print('Prima(2)=', evolucion[1])
print('Prima(3)=', evolucion[2])
print('Prima(4)=', evolucion[3])
print('Prima('+str(m)+')=', evolucion[m-1])
print()
BS = primaBS('call', 100, 110, 2, 0.2, 0.07)
print('Prima Black-Scholes=', BS)
print()

In [ ]:
# Analizamos la convergencia entre ambos métodos
# y representamos el gráfico de convergencia
# Se comprueba que cuando el número de pasos del método binomial tiende a infinito 
# la diferencia entre el valor teórico obtenido y el valor que proporciona el 
# método de Black-Scholes tiende a 0.


m = 200                                   
diferencia = [0]*(m)
for j in range(m):
    diferencia[j] = evolucion[j] - BS
   # print(j+1, evolucion[j], BS, diferencia[j])


fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
last = 1
bound = m
ax.plot(diferencia)
fig.savefig('graph.png')